In [1]:
import datetime
import os
import shutil
import stat
import subprocess
import tempfile
import time
import requests

In [2]:
def get_user_repositories(username):
    url = f"https://api.github.com/users/{username}/repos"
    response = requests.get(url)

    if response.status_code == 200:
        repositories = response.json()
        public_repos = [repo for repo in repositories if repo['private']==False]
        urls = [repo['url'].replace('https://api.github.com/','https://github.com/').replace('/repos/','/') + '.git' 
                for repo in public_repos]
        return urls
    else:
        print(f"Failed to fetch repositories for user {username}. Status code: {response.status_code}")
        return None

In [15]:
def get_commit_emails_and_dates(repo_url,fullname,username,email_date_dict = {},verbose=False):
    try:
        temp_dir = tempfile.mkdtemp()
        if verbose:
            print(f'pulling from {repo_url}... \ntemp dir is {temp_dir}')
        # Clone the repository into the temporary directory
        result = subprocess.run(['git', 'clone', repo_url, temp_dir], capture_output=True, text=True)

        if result.returncode != 0:
            print(f"Error cloning repository: {result.stderr}")
            return None

        os.chdir(temp_dir)
        process = subprocess.Popen(['git', 'log', '--pretty=format:%an|%ae|%cd', '--date=iso'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout, stderr = process.communicate()
        
        if process.returncode != 0:
            print(f"Error getting commit log: {stderr}")
            return None

        commits = stdout.splitlines()
        default_date = datetime.date(1970, 1, 1)
        for commit in commits:
            commit_name, email, date_str = commit.split('|')
            if commit_name not in [fullname, username]:
                continue
            
            if '@' not in email:
                continue
                
            if '-' in email.split('@',1)[1]:
                continue
                
            if email.endswith('.local') or email.endswith('.lan'):
                continue

            date_log = datetime.datetime.strptime(date_str[:10],'%Y-%M-%d').date()
            if date_log > email_date_dict.get(email,default_date):
                email_date_dict[email] = date_log
        subprocess.run(['cd', '..'], shell=True, check=True)
    except Exception as e:
        print(f'Error pulling from {repo_url}: {e}')
    finally:
        return email_date_dict, temp_dir

In [16]:
def get_fullname(username):
    url = f"https://api.github.com/users/{username}"
    response = requests.get(url)

    if not response.ok:
        raise RuntimeError(f'Error pulling username. {response.status_code}: {response.reason}')
    user_dict = response.json()

    if (fullname :=user_dict.get('name')):
        return fullname
    else:
        raise RuntimeError('No Name for Username')

In [17]:
def delete_temp(temp_dir):
    '''
    Changes permission of all files in temp_dir. Some are read-only.
    '''
    os.chdir(temp_dir)
    os.chdir('..')
    for root, dirs, files in os.walk(temp_dir):  
        for dir in dirs:
            os.chmod(os.path.join(root, dir), stat.S_IRWXU)
        for file in files:
            os.chmod(os.path.join(root, file), stat.S_IRWXU)
    shutil.rmtree(temp_dir)

In [18]:
def main():
    username = input('Enter username (from URL): ')
    fullname = get_fullname(username)
    urls = get_user_repositories(username)
    if not urls:
        raise ValueError('no public repos to pull from')

    email_date_dict = {}
    for url in urls:
        email_date_dict,temp_dir = get_commit_emails_and_dates(url,fullname,username,email_date_dict)
        delete_temp(temp_dir)

    email_date_tuples = [(email,date) for email,date in email_date_dict.items()]
    email_date_tuples.sort(key = lambda x:x[1],reverse=True)

    print(f'\n\n\nEmails for {fullname}:')
    for email,max_date in email_date_tuples:
        max_date_formatted = max_date.strftime('%Y-%m-%d')
        print(f'{email} - {max_date_formatted}')

In [19]:
if __name__ == '__main__':
    main()

Enter username (from URL): KishPatel1996


Exception in thread Thread-371:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\ProgramData\Anaconda3\lib\subprocess.py", line 1479, in _readerthread
    buffer.append(fh.read())
  File "C:\ProgramData\Anaconda3\lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 13574: character maps to <undefined>


Error pulling from https://github.com/KishPatel1996/react-native.git: 'NoneType' object has no attribute 'splitlines'



Emails for Kishan Patel:
kishan.patel@berkeley.edu - 2020-01-07
kishcoin@gmail.com - 2015-01-27
kishan.patel.271996@gmail.com - 2015-01-18


In [ ]:
KishPatel1996